In [1]:
import pandas as pd

In [4]:
train_df = pd.read_csv('./Datawhale_学术论文分类_数据集/train.csv', sep='\t')
test_df = pd.read_csv('./Datawhale_学术论文分类_数据集/test.csv', sep='\t')

In [9]:
train_df.head()

,paperid,title,abstract,categories
0,train_00000,"Hard but Robust, Easy but Sensitive: How Encod...",Neural machine translation (NMT) typically a...,cs.CL
1,train_00001,An Easy-to-use Real-world Multi-objective Opti...,Although synthetic test problems are widely ...,cs.NE
2,train_00002,Exploration of reproducibility issues in scien...,This is the first part of a small-scale expl...,cs.DL
3,train_00003,Scheduled Sampling for Transformers,Scheduled sampling is a technique for avoidi...,cs.CL
4,train_00004,Hybrid Forests for Left Ventricle Segmentation...,Machine learning models produce state-of-the...,cs.CV


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

tfidf = TfidfVectorizer(max_features=500)

In [33]:
# 只是用标题
train_tfidf = tfidf.fit_transform(train_df['title'])

clf = SGDClassifier()
cross_val_score(clf, train_tfidf, train_df['categories'], cv=5)

array([0.49001398, 0.48456389, 0.48075577, 0.49429544, 0.48437187])

In [34]:
# 使用摘要
train_tfidf = tfidf.fit_transform(train_df['abstract'])

clf = SGDClassifier()
cross_val_score(clf, train_tfidf, train_df['categories'], cv=5)

array([0.61074496, 0.61574583, 0.60811756, 0.61068855, 0.61671008])

In [35]:
# 标题和摘要
train_df['title_abstract'] = train_df['title'] + ' ' + train_df['abstract']
train_tfidf = tfidf.fit_transform(train_df['title_abstract'])

clf = SGDClassifier()
cross_val_score(clf, train_tfidf, train_df['categories'], cv=5)

array([0.62452566, 0.62793486, 0.62171349, 0.62580064, 0.63103586])

In [40]:
test_df['title_abstract'] = test_df['title'] + ' ' + test_df['abstract']
test_tfidf = tfidf.transform(test_df['title_abstract'])

clf = SGDClassifier()
clf.fit(train_tfidf, train_df['categories'])
test_df['categories'] = clf.predict(test_tfidf)

In [41]:
test_df[['paperid', 'categories']].to_csv('submit2.csv', index=None)